In [1]:
import cobra as cb
import tempfile
from pathlib import Path

In [ ]:
!wget 'http://bigg.ucsd.edu/static/models/iML1515.xml'

In [2]:
data_path = Path.cwd().resolve()

In [3]:
chassis = cb.io.read_sbml_model(data_path.joinpath('iML1515.xml'))

## Step 1
Generate the metabolites the new pathway will use:

In [4]:
pet_e = cb.Metabolite('pet_e', name='polyethylene terephthalate', compartment='e')
pet_minus_e = cb.Metabolite('petm_e', name='PET(n-1)', compartment='e') # pseudo metabolite for balancing reactions
mhet_e = cb.Metabolite('mhet_e', name='2-Hydroxyethyl terephthalic acid', compartment='e')
tpa_e = cb.Metabolite('tpa_e', name='Terephthalic Acid', compartment='e',
                      formula='C8H6O4')
glycol_e = cb.Metabolite('glycol_e', name='Ethylene Glycol, 1,2-ethanediol', compartment='e',
                         formula='C2H6O2')
glycol_c = cb.Metabolite('glycol_c', name='Ethylene Glycol, 1,2-ethanediol', compartment='c',
                         formula='C2H6O2')


## Step 2
Create the reactions the pathway will use:

In [5]:
# PET depolymerization
petase = cb.Reaction('petase', name='PET Hydrolase', subsystem='PET metabolism',
                     lower_bound= 10.0, upper_bound=1000.0)
petase.add_metabolites({pet_e:-1, chassis.metabolites.get_by_id('h2o_e'):-1,
                        pet_minus_e:1, mhet_e:1})

# Breakdown of PET monomer (MHET) into TPA and ethylene glycol
mhetase = cb.Reaction('mhetase', name='MHET Acylhydrolase', subsystem='PET metabolism',
                      lower_bound=100.0, upper_bound=1000.0)
mhetase.add_metabolites({mhet_e:-1, chassis.metabolites.get_by_id('h2o_e'):-1,
                         tpa_e:1, glycol_e:1})

# Ethylene glycol uptake into cell
gly_diff = cb.Reaction('gly_diff', name='Ethylene Glycol transmembrane diffusion',
                        lower_bound=-1000.0, upper_bound=1000.0)
gly_diff.add_metabolites({glycol_e:-1, glycol_c:1})

# Ethylene glycol oxidization to glycolaldehyde--contact with main metabolic network
gly_oxidase = cb.Reaction('gly_oxidase', name='Ethylene Glycol NAD+ dependent oxidoreductase',
                          lower_bound=-100.0, upper_bound=1000.0)
gly_oxidase.add_metabolites({glycol_c:-1, chassis.metabolites.get_by_id('gcald_c'):1})

## Step 3
Add the reactions to the main model:

In [6]:
reactions = [petase, mhetase, gly_diff, gly_oxidase]
chassis.add_reactions(reactions)

## Step 4
Add boundary reactions, or unbalanced pseudoreactions which add or remove metabolites from the model

PET needs to be input to the model, the PET minus pseudometabolite needs to be removed, and TPA needs to be removed,
simulating harvesting for industrial use:

In [7]:
chassis.add_boundary(pet_e, type='exchange')

Reaction identifier,EX_pet_e
Name,polyethylene terephthalate exchange
Memory address,0x7f1b96d33850
Stoichiometry,pet_e <=> polyethylene terephthalate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [8]:
chassis.add_boundary(pet_minus_e, type='exchange')

Reaction identifier,EX_petm_e
Name,PET(n-1) exchange
Memory address,0x7f1b96d32d10
Stoichiometry,petm_e <=> PET(n-1) <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [9]:
chassis.add_boundary(tpa_e, type='exchange')

Reaction identifier,EX_tpa_e
Name,Terephthalic Acid exchange
Memory address,0x7f1b96d33dc0
Stoichiometry,tpa_e <=> Terephthalic Acid <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Step 5
Calculate optimal flux balance solution:

In [10]:
chassis_solution = chassis.optimize()

In [11]:
chassis_solution

,fluxes,reduced_costs
CYTDK2,0.000000,8.609565e-18
XPPT,0.000000,1.387779e-17
HXPRT,0.000000,-3.496001e-02
NDPK5,0.000000,2.409850e-17
SHK3Dr,5.514906,1.535811e-17
...,...,...
gly_diff,1000.000000,0.000000e+00
gly_oxidase,1000.000000,-8.673617e-19
EX_pet_e,-1000.000000,0.000000e+00
EX_petm_e,1000.000000,0.000000e+00


In [12]:
for reaction in reactions:
    print(f'{reaction.name} flux: {reaction.flux}')

PET Hydrolase flux: 1000.0000000000001
MHET Acylhydrolase flux: 1000.0000000000001
Ethylene Glycol transmembrane diffusion flux: 1000.0
Ethylene Glycol NAD+ dependent oxidoreductase flux: 1000.0


## Results

We can see here that the PET degradation pathway is active to the maximum degree possible during flux balance analysis. This indicates that, at least from a chemical point of view, this metabolism is advantageous to the organism and is likely to be retained. This is expected, given that it not only acts as a carbon source but also reduces NAD+ to NADH, resulting in an energy gain for the cell.

In [13]:
cb.io.write_sbml_model(chassis, str(data_path.joinpath('iML1515_aug.xml')))

In [14]:
cb.io.save_json_model(chassis, str(data_path.joinpath('iML1515_aug.json')))

In [15]:
chassis.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.07532,0,0.00%
cl_e,EX_cl_e,0.07532,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0003618,0,0.00%
cu2_e,EX_cu2_e,0.01026,0,0.00%
fe2_e,EX_fe2_e,0.2324,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
h2o_e,EX_h2o_e,1000,0,0.00%
k_e,EX_k_e,2.825,0,0.00%
mg2_e,EX_mg2_e,0.1255,0,0.00%
mn2_e,EX_mn2_e,0.009999,0,0.00%


## Results
In this example, the model has been instructed to maximize microbial biomass (simulating
the bacteria's primary goal to grow and reproduce). PET metabolism was not included in the
objective function. Despite this, both degradation of PET and secretion of TPA occur at their maximum
possible rates. This suggests that this pathway would be highly active *in vivo*.

Recall that for PET and the PET(n-1) pseudovariable we left the chemical formula blank.
This is why cobra does not report carbon fluxes for these metabolites.